In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from math import cos, degrees, pi, log, radians, ceil
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
%matplotlib notebook

from matplotlib import colors as clr
import matplotlib

def new_nei_set(alpha, cols_bin, perc):
    return np.flatnonzero(cols_bin[alpha]>perc)

In [2]:
cl=[(1.0,0.0,0.0,1),
(1.0,1.0,0.0,1),
(0.0,1.0,0.0,1),
(0.0,1.0,1.0,1),
(153/255,51/255,1.0,1),
(1,51/255,1,1), (0.0,0.0, 1.0, 1), (1.0,0.0,1.0,1), (0.0,0.5,0.5,1), (192/255, 192/255,192/255,1) ]   

## TOC:
* [Calculation of optimal angles](#first-bullet)
* [Greedy algorithm](#second-bullet)
* [Preliminary segmentation](#third-bullet)
* [Two sets](#fourth-bullet)
* [Three sets](#fifth-bullet)

In [ ]:
# It is supposed that the data is preprocessed according to the characteristics of the instrument and data acquisition parameters, then binned and put into the following variables 
# vv -- a 2d numpy.array of intensities, vv[i] is the array of intensities of the point number i
# xx and yy -- coordinates in the the image. x[i] and y[i] are the coordinates of the point number i

# Calculation of optimal angles <a class="anchor" id="first-bullet"></a>

In [ ]:
number0=len(vv)
cols_bin=1-pairwise_distances(vv, vv, metric="cosine")
min_angle=np.arccos(np.sort(cols_bin, axis=None)[-number0-1])
print("Minimal angle is ", min_angle, " radian")

In [ ]:
%%time
# alpha -- an array of densities, extr_phi -- an array of optimal angles
#delta_phi=min_angle/10
delta_phi=0.01
print(int(ceil(1/delta_phi)))

alpha1=[]
extr_phi1=[]
logx=np.array([degrees(i*delta_phi) for i in range(3, int(ceil(1/delta_phi)))])
for i in range(number0):
    if(i%10000==0):
        print(i)
    logy=np.array([(log(len(new_nei_set(i, cols_bin, cos(k*delta_phi)))))/log(k) for k in range (3, int(ceil(1/delta_phi)))], dtype=float)
    alpha1.append(np.amax(logy))
    extr_phi1.append(degrees((np.argmax(logy)+2)*delta_phi))
alpha1=np.array(alpha1)
extr_phi1=np.array(extr_phi1)

In [ ]:
# Picture
fig5, ax= plt.subplots(figsize=(8,8))
ax.scatter(alpha1, extr_phi1, s=1)
ax.set_xlabel("Density")
ax.set_ylabel("Max angle")
plt.show()

In [ ]:
# Filtering the outliers
z=np.flatnonzero(extr_phi1>45)
print("The number of outliers is equal to ", len(z))
ileft= np.array(list({j for j in range(number0)}.difference(set(z))))
x=xx[ileft]
y=yy[ileft]
v=np.array([vv[ileft[i]] for i in range(len(ileft))])
alpha=alpha1[ileft]
extr_phi=extr_phi1[ileft]

In [ ]:
# Recalculation CSM
del(cols_bin)
cols_bin=1-pairwise_distances(v, v, metric="cosine")
mincos=np.amin(cols_bin, axis=None)
number=len(v)

In [ ]:
# Clearing the variables
del(xx)
del(yy)
del(vv)

In [ ]:
# For future pictures
extent = np.min(x), np.max(x), np.min(y), np.max(y)
xgrid=np.arange(extent[0]-0.5, extent[1]+0.5, 1)
ygrid=np.arange(extent[2]-0.5, extent[3]+0.5, 1)

Z0=[[(0.0,0.0,0.0,1.0) for j in range(len(xgrid))] for i in range(len(ygrid))]

# Greedy algorithm <a class="anchor" id="second-bullet"></a>

In [ ]:
# This "greedy" algorithm chooses the "best" set at each step without redefining the previous ones. At the first step Take the angle phi corresponding to the maximal value of alpha.
# The "best" set is defined in the following ways:
# 1) (key=2)  Each cone has the apex angle chosen manually. (cosine is equal to cos_opt)
#At every next step choose the biggest set among the cones with the apex angle phi not intersecting with all previous.
# 2) (key=1) Each cone has the apex angle equal to the corresponding to extr_phi. 
#Choose the cone corresponding to the point with the biggest alpha, such that the set in the cone doesn't intersect with all previous.

In [ ]:
key=1 #different angles
#key=2 #the same angle

In [ ]:
mincos=np.amin(cols_bin, axis=None)
num_opt=np.argmax(alpha)
med_phi=np.median(extr_phi) #median angle among all extr_phi

In [ ]:
# if key=2, choose one of the following
#cos_opt=cos(radians(extr_phi[num_opt])) #the angle corresponding to the biggest alpha
#cos_opt=cos(radians(np.median(extr_phi))) #median angle
cos_opt=cos(np.arccos(mincos)/10) #max angle among all the pairwise angles/10

In [ ]:
%%time
# cone_matr -- is a matrix of 0 and 1: take the cone with the apex angle according to key (see above), 
# 0 in the cell (i,j) means that j-th point belongs to the i-th cone, 1 - otherwise
# len_list -- the number of points in each cone
len_list=np.zeros(number, dtype=int)
cone_matr=np.ones((number, number), dtype=bool) 
        
if(key==2):
    for i in range(number):
        temp=new_nei_set(i, cols_bin, cos_opt)
        len_list[i]=len(temp)
        for j in temp:
            cone_matr[i][j]=0
elif (key==1):
    for i in range(number):
        temp=new_nei_set(i, cols_bin, cos(radians(extr_phi[i])))
        len_list[i]=len(temp)
        for j in temp:
            cone_matr[i][j]=0

In [ ]:
%%time

nums=np.array([num_opt])
temp=np.prod(cone_matr.T[np.flatnonzero(cone_matr[num_opt]==0)], axis=0) # temporary set
good_nums=np.flatnonzero(temp) #numbers of points left to consider

while len(good_nums)>0:
    if(key==1):
        num_t=good_nums[np.argmax(alpha[good_nums])]
    elif(key==2):
        num_t=good_nums[np.argmax(len_list[good_nums])]
    nums=np.append(nums, num_t)
    temp=temp*np.prod(cone_matr.T[np.flatnonzero(cone_matr[num_t]==0)], axis=0)
    good_nums=np.flatnonzero(temp)
    if(len_list[num_t]==1): #don't consider the cones with 1 point inside, as any cone contains it's vertex
        break

In [ ]:
# Picture of 9 biggest sets
cols4=np.array([cl[-1] for i in range(number)])

if(key==2):
    for j in range(min(len(nums), 10)):
        for i in new_nei_set(nums[j], cols_bin, cos_opt):
            cols4[i]=cl[j]    
elif (key==1):
    for j in range(min(len(nums), 10)):
        for i in new_nei_set(nums[j], cols_bin, cos(radians(extr_phi[nums[j]]))):
            cols4[i]=cl[j]
            
Z2=Z0

for i in range(len(x)):
    Z2[extent[3]-y[i]][x[i]-extent[0]]=cols4[i]

fig1 = plt.figure(figsize=(8,8), frameon=False)
im1 = plt.imshow(Z2, extent=extent)
plt.show()

# Preliminary calculations <a class="anchor" id="third-bullet"></a>

In [ ]:
# Type 0 in reg_num for the median angle for zero approximation, or 2 or 3 for Phi/4 and Phi/6 respectively.
reg_num=0
if(reg_num==0):
    coneangle=cos(radians(np.median(extr_phi)))
elif(reg_num>0):
    coneangle=cos(np.arccos(mincos)/2/reg_num)
else:
    print("There is an error")

print("Maximal angle is "+str(degrees(np.arccos(mincos)))+" degrees")
print("Number of points in the image is "+str(number))

In [ ]:
%%time
#arrofnei_new -- matrix number*number, Matrix of neighbours: if 1 then not neigbours, if 0 then are neighbours
#arr_of_notnei -- matrix number*number, Matrix of not-neighbours of all the neighbours. If 1 then can be an axis of the other cone
lenlist=np.zeros(number, dtype=int)
arrofnei_new=np.ones((number, number), dtype=bool)
for i in range(number):
    nei_temp=new_nei_set(i, cols_bin, coneangle)
    for j in nei_temp:
        arrofnei_new[i][j]=0
    lenlist[i]=len(nei_temp)
arr_of_notnei=np.array([np.prod(arrofnei_new.T[np.flatnonzero(arrofnei_new[j]==0)], axis=0) for j in range(number)])  

# Two sets <a class="anchor" id="fourth-bullet"></a>

In [ ]:
%%time
#Preliminary segmentation with angle = coneangle
lentemp=0
for i in range(number):
    set0=np.flatnonzero(arr_of_notnei[i][i:])+i
    if(len(set0)):
        answ1=np.argmax(lenlist[set0])
        answ2=set0[answ1]
        if(lenlist[i]+lenlist[answ2]>lentemp):
            bestnum1=i
            bestnum2=answ2
            lentemp=lenlist[i]+lenlist[answ2]
arr_answ=np.array([bestnum1, bestnum2])

In [ ]:
#Picture for 2 sets
cols7=np.array([cl[-1] for i in range(number)])
fs1=np.flatnonzero(arrofnei_new[bestnum1]==0)
fs2=np.flatnonzero(arrofnei_new[bestnum2]==0)
for i in fs1: 
    cols7[i]=cl[1]
for i in fs2:
    cols7[i]=cl[0]


Z2=Z0

for i in range(len(x)):
    Z2[int(extent[3]-y[i])][int(x[i]-extent[0])]=cols7[i]

fig1 = plt.figure(figsize=(8,8), frameon=False)
im1 = plt.imshow(Z2, extent=extent)
plt.show() 

In [ ]:
%%time
# set_of_sets -- matrix of dimensions num_of_reg*number, a 0 in (i,j) cell means that the j-th point belongs to i-th cone
# phi_set -- the set of angles (will be needed to draw a picture)
num_of_reg=2
set_of_sets=arrofnei_new[arr_answ]
phi_set=np.array([cos(radians(np.median(extr_phi[np.flatnonzero(set_of_sets[k-1]==0)])))  for k in range(num_of_reg)])

new_lenlist=np.zeros(number, dtype=int)
setlist=np.zeros((num_of_reg, number), dtype=bool) # 1 in the (i,j) cell means that the point j belongs to the i--th set
nei_arr=np.ones((number, number), dtype=bool) # if i belongs to Set_j, then 0  at (i,k)-cell means that k-th point belongs to the cone with i as axis and phi_j as the apex angle 
#If i doesn't belong to any Set_j, then all the line consists of 1s only

for k in range(num_of_reg):
    Set_temp=np.flatnonzero(arrofnei_new[arr_answ[k]]==0) #Numbers of points in the Set_k
    for i in (Set_temp):
        setlist[k][i]=1
        nei_temp=new_nei_set(i, cols_bin, phi_set[k])
        for j in nei_temp:
            nei_arr[i][j]=0
        new_lenlist[i]=number-np.sum(nei_arr[i])

In [ ]:
%%time
not_nei_arr=np.array([np.prod(nei_arr.T[np.flatnonzero(nei_arr[j]==0)], axis=0) for j in range (number)], dtype=bool)

In [ ]:
%%time
# Two sets refined
set0=np.flatnonzero(setlist[0]) #numbers of points in Set_1
fin_ans=np.zeros(2, int)
jlen=0
for i in set0:
    set1=np.flatnonzero(not_nei_arr[i]*setlist[1]) #numbers of those having no neighbours with i-th and belonging to the Set_2
    if(len(set1)):
        answ1=np.argmax(new_lenlist[set1])
        answ2=set1[answ1]
        if(new_lenlist[answ2]+new_lenlist[i]>jlen): 
            fin_ans[0]=i
            fin_ans[1]=answ2
            jlen=new_lenlist[answ2]+new_lenlist[i]

In [ ]:
final_set1=np.flatnonzero(nei_arr[fin_ans[0]]==0)
final_set2=np.flatnonzero(nei_arr[fin_ans[1]]==0)

In [ ]:
#Picture for 2 sets
cols6=np.array([cl[-1] for i in range(number)])
for i in final_set1: 
    cols6[i]=cl[1]
for i in final_set2:
    cols6[i]=cl[0]


Z2=Z0

for i in range(len(x)):
    
    Z2[int(extent[3]-y[i])][int(x[i]-extent[0])]=cols6[i]

fig1 = plt.figure(figsize=(8,8), frameon=False)
im1 = plt.imshow(Z2, extent=extent)
plt.show() 

# Three sets <a class="anchor" id="fifth-bullet"></a>

In [ ]:
%%time
#Preliminary segmentation with angle = coneangle
lentemp=0
for i in range(number):
    set0=np.flatnonzero(arr_of_notnei[i][i:])+i
    for j in set0:
        set1=np.flatnonzero((arr_of_notnei[i]*arr_of_notnei[j])[j:])+j            
        if(len(set1)):
            answ1=np.argmax(lenlist[set1])
            answ2=set1[answ1]
            if(lenlist[i]+lenlist[j]+lenlist[answ2]>lentemp):
                bestnum1=i
                bestnum2=j
                bestnum3=answ2
                lentemp=lenlist[i]+lenlist[j]+lenlist[answ2]
arr_answ=np.array([bestnum1, bestnum2, bestnum3])

In [ ]:
%%time
# set_of_sets -- matrix of dimensions num_of_reg*number, a 0 in (i,j) cell means that the j-th point belongs to i-th cone
# phi_set -- the set of angles (will be needed to draw a picture)
num_of_reg=3
set_of_sets=arrofnei_new[arr_answ]
phi_set=np.array([cos(radians(np.median(extr_phi[np.flatnonzero(set_of_sets[k-1]==0)])))  for k in range(num_of_reg)])

new_lenlist=np.zeros(number, dtype=int)
setlist=np.zeros((num_of_reg, number), dtype=bool) # 1 in the (i,j) cell means that the point j belongs to the i--th set
nei_arr=np.ones((number, number), dtype=bool) # if i belongs to Set_j, the in the i-th line then 0 in (i,) means that k-th point belongs to the cone with i as axis and phi_j as the apex angle 
#If i doesn't belong to any Set_j, then all the line consists of 1s only

for k in range(num_of_reg):
    Set_temp=np.flatnonzero(arrofnei_new[arr_answ[k]]==0) #Numbers of points in the Set_k
    for i in (Set_temp):
        setlist[k][i]=1
        nei_temp=new_nei_set(i, cols_bin, phi_set[k])
        for j in nei_temp:
            nei_arr[i][j]=0
        new_lenlist[i]=number-np.sum(nei_arr[i])

In [ ]:
%%time
not_nei_arr=np.array([np.prod(nei_arr.T[np.flatnonzero(nei_arr[j]==0)], axis=0) for j in range (number)], dtype=bool)

In [ ]:
%%time
# Three sets refined
fin_ans3=np.zeros(3, int)
set0=np.flatnonzero(setlist[0])
jlen=0
for i in set0:
    set1=np.flatnonzero(not_nei_arr[i]*setlist[1])
    for j in set1:
        set2=np.flatnonzero(not_nei_arr[j]*not_nei_arr[i]*setlist[2]) #having no neighbours with ith and with j-th and belonging to the Set_3
        if(len(set2)):
            answ1=np.argmax(new_lenlist[set2])
            answ2=set2[answ1]
            if(new_lenlist[answ2]+new_lenlist[i]+new_lenlist[j]>jlen):
                fin_ans3[0]=i
                fin_ans3[1]=j
                fin_ans3[2]=answ2    
                jlen=new_lenlist[answ2]+new_lenlist[i]+new_lenlist[j]

In [ ]:
final_set1=np.flatnonzero(nei_arr[fin_ans3[0]]==0)
final_set2=np.flatnonzero(nei_arr[fin_ans3[1]]==0)
final_set3=np.flatnonzero(nei_arr[fin_ans3[2]]==0)

In [ ]:
#Picture for three sets
cols5=np.array([cl[-1] for i in range(number)])
for i in final_set1: 
    cols5[i]=cl[0]
for i in final_set2:
    cols5[i]=cl[1]
for i in final_set3:
    cols5[i]=cl[2]

Z2=Z0

for i in range(len(x)):
   Z2[int(extent[3]-y[i])][int(x[i]-extent[0])]=cols5[i]

fig1 = plt.figure(figsize=(8,8), frameon=False)
im1 = plt.imshow(Z2, extent=extent)
plt.show() 